## Answering Business Questions using SQL

### 1. Introduction and Schema Diagram

In this project, we are using the Chinook database,  is provided as a SQLite database file called `chinook.db`. A copy of the database schema is below.

![chinook.db](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

We are going to use SQL queries to analyse the following: 

* The best selling genre in USA
* Performance of the sales representative 
* Sales by country
* Purchase of albums vs purchase of individual tracks
* Which artist is used in the most playlists?
* How many tracks have been purchased vs not purchased?
* Is the range of tracks in the store reflective of their sales popularity?
* Do protected vs non-protected media types have an effect on popularity?

Let's connect Jupyter Notebook to the database file and get started with our analysis. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db
    

'Connected: None@chinook.db'

### 2. Overview of the database tables and views

In [2]:
%%sql 
SELECT name, type from sqlite_master WHERE type IN("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [18]:
%%sql 
select *from album LIMIT 10;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3
6,Jagged Little Pill,4
7,Facelift,5
8,Warner 25 Anos,6
9,Plays Metallica By Four Cellos,7
10,Audioslave,8


### 3. Genre and number of tracks sold in USA

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce as in the image below. 

![artist_genres](artist_genre_table.png)

Let's query out which genres sell the best in the USA.


In [4]:
%%sql
WITH usa AS 
(SELECT il.*
FROM  invoice_line il 
LEFT JOIN invoice i ON  i.invoice_id = il.invoice_id
LEFT JOIN customer c ON c.customer_id = i.customer_id
WHERE i.billing_country = 'USA')
SELECT g.name "genre",
SUM(usa.quantity)"tracks_sold" ,
CAST(COUNT(usa.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa
    ) "percentage_sold"
FROM usa usa
LEFT JOIN track t ON t.track_id = usa.track_id
LEFT JOIN genre g ON g.genre_id = t.genre_id

GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


From the list of four, the following can be added to the store based of the results in the above cell.

1. Punk by Regal
2. Blues by Slim Jim Bites
3. Pop by Red Tone
However, the total percent sold for the above 3 genres is equal to  approx. 17% which is far lesser than genre Rock that tops the list with more than 50% sales in USA.  It is recommended that we purchase tracks in the Rock genre for the store.


### 4. Analysing performance of Sales Agents

In [5]:
%%sql

select e.first_name||' '||e.last_name "Sales Rep Name",
e.hire_date "Hire date",
ROUND(SUM(il.unit_price* il.quantity),2) "Total amount of sales" from invoice i
inner join invoice_line il on il.invoice_id = i.invoice_id 
inner join customer c on i.customer_id = c.customer_id
inner join employee e on e.employee_id = c.support_rep_id
group by 1
order by 3 desc;

Done.


Sales Rep Name,Hire date,Total amount of sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


There is approx. 20% (19.49%) difference in the sales between  Jane(the best sales employee) and Steve(the least performing). 

### 5. Analyzing Sales by Country

In [6]:
%%sql
WITH countries as 
(
    SELECT CASE WHEN
    (
        SELECT COUNT(*) from customer
        WHERE c.country = country
    ) = 1 THEN "Other"
    ELSE c.country END "Country", c.customer_id,i.*
    FROM invoice i 
    INNER JOIN  customer c  on i.customer_id = c.customer_id
   
)
SELECT country, total_customers, total_sales, avg_sales_per_customer, avg_order_value
FROM (
    SELECT country, COUNT(distinct customer_id)"total_customers",
    SUM(total)/COUNT(distinct invoice_id)"avg_order_value",
    SUM(total)"total_sales", SUM(total)/COUNT(distinct customer_id)"avg_sales_per_customer",
    CASE
        WHEN country = "Other" THEN 1
        ELSE 0
        END AS sort
    FROM countries
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
     
)



Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


### 6. Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

   *  purchase a whole album
   * purchase a collection of one or more individual tracks.
   
The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Let's find out what percentage of purchases are individual tracks vs whole albums.


In [11]:
%%sql
WITH detailed_invoice AS
    (
        SELECT t.track_id, i.invoice_id
            FROM track t
            INNER JOIN invoice_line il ON il.track_id = t.track_id
            INNER JOIN invoice i ON i.invoice_id = il.invoice_id
            GROUP BY i.invoice_id
    )
SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    ROUND(CAST(COUNT(invoice_id) AS FLOAT)/(SELECT COUNT(*)
                                            FROM invoice)*100 ,2)"percentage_of_invoices"
FROM
(
    SELECT di.*,
        CASE
            WHEN(
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = di.track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = di.invoice_id
                 ) IS NULL
             AND (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = di.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = di.track_id
                                     ) 
                 ) IS NULL
             THEN "YES"
             ELSE "NO"
         END AS "album_purchase"
     FROM detailed_invoice di
    )
GROUP BY album_purchase;


Done.


album_purchase,number_of_invoices,percentage_of_invoices
NO,500,81.43
YES,114,18.57


### 7. Further analysis

#### Which artist is used in the most playlists?

In [13]:
%%sql

SELECT ar.name "artist", count(p.playlist_id)"count" ,p.name"playlist", g.name"genre"
FROM  playlist_track pt
LEFT JOIN track t  ON pt.track_id = t.track_id
LEFT JOIN album al ON al.album_id = t.album_id
LEFT JOIN artist ar ON ar.artist_id = al.artist_id
LEFT JOIN genre g ON t.genre_id = g.genre_id
LEFT JOIN playlist p ON p.playlist_id = pt.playlist_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Done.


artist,count,playlist,genre
Iron Maiden,516,Heavy Metal Classic,Metal
U2,333,Music,Pop
Metallica,296,Heavy Metal Classic,Metal
Led Zeppelin,252,Music,Rock
Deep Purple,226,Music,Rock
Lost,184,TV Shows,Drama
Pearl Jam,177,Grunge,Rock
Eric Clapton,145,Brazilian Music,Latin
Faith No More,145,Music,Alternative & Punk
Lenny Kravitz,143,Music,Metal


Iron Maiden has been used in most playlists.

#### How many tracks have been purchased vs not purchased?

In [14]:
%%sql
 WITH invoiced AS 
    (
        SELECT  track_id  FROM invoice_line
    )
        
SELECT COUNT(DISTINCT p.track_id) as tracks_purchased, 
(
    SELECT COUNT(DISTINCT track_id)
    FROM track 
    WHERE track_id NOT IN (SELECT track_id FROM invoiced)
) AS tracks_not_purchased
FROM invoiced p  
LEFT JOIN track t ON t.track_id = p.track_id

 




Done.


tracks_purchased,tracks_not_purchased
1806,1697


1806 distinct tracks have been purchased and 1697 were not purchased from the store.

#### Do protected vs non-protected media types have an effect on popularity?

In [15]:
%%sql
select * from media_type

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


`media_type_id` 2 and 3 are protected and the rest are not. Now let's proceed to query out the sales popularity based on the media type.

In [16]:
%%sql

WITH protected AS
(
    SELECT SUM(i.unit_price * i.quantity)"price", i.track_id,t.media_type_id
    FROM invoice_line i
    INNER JOIN track t ON t.track_id = i.track_id
    WHERE t.media_type_id = 2 or t.media_type_id = 3
),
unprotected AS 
(
    SELECT SUM(i.unit_price * i.quantity)"price", i.track_id,t.media_type_id 
    FROM invoice_line i
    INNER JOIN  track t ON t.track_id = i.track_id
    WHERE t.media_type_id != 2 and t.media_type_id != 3
)

SELECT SUM(p.price) AS protected, SUM(u.price) AS unprotected 
FROM media_type m
LEFT JOIN  protected p  ON m.media_type_id = p.media_type_id
LEFT JOIN  unprotected u ON m.media_type_id = u.media_type_id



Done.


protected,unprotected
437.58000000000254,4271.849999999527


We see that the unprotected media type generated more sales than the protected media.


#### Is the range of tracks in the store reflective of their sales popularity?

Let's see the popularity of the tracks based on the genre.


In [17]:
%%sql

WITH invoiced AS
(
    SELECT i.unit_price , i.quantity, i.track_id,t.genre_id  
     FROM invoice_line i
     INNER JOIN track t ON t.track_id = i.track_id 
)
SELECT   g.name "genre",
ROUND(SUM(i.quantity)/(
                       SELECT CAST(sum(quantity) AS FLOAT) 
                         FROM invoice_line)*100 ,2)"popularity_percent", 
ROUND(SUM(i.unit_price * i.quantity) ,2)"invoiced_amount" 

FROM invoiced i
LEFT JOIN genre g ON i.genre_id = g.genre_id
 

GROUP BY 1 
ORDER BY 2 DESC ,3 DESC
LIMIT 10

Done.


genre,popularity_percent,invoiced_amount
Rock,55.39,2608.65
Metal,13.01,612.81
Alternative & Punk,10.34,487.08
Latin,3.51,165.33
R&B/Soul,3.34,157.41
Blues,2.61,122.76
Jazz,2.54,119.79
Alternative,2.46,115.83
Easy Listening,1.56,73.26
Pop,1.32,62.37


The range of tracks based on different genre do shows that tracks under genre Rock the most popular and is brought frequently followed by genre Metal. The range of tracks in the store do have a major role in the sales popularity of the store


### Conclusion

In this project we used SQL queries to answer business questions using the Chinook database.
